In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip data.zip
!unzip model.zip
!unzip util.zip

In [1]:
import os
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import torch
import torch.nn as nn
import random
import torch.optim as optim
import utils
import skimage.color as sc
import datetime
import pandas as pd
from data.CustomDataset import imageDataset
from torch.utils.data import DataLoader, random_split
from torchvision import transforms as T
from model import esrt
from EntropyL1 import EntropyL1

/home/rts3012/anaconda3/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_cuda.so: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
LEARNING_RATE = 2e-4
SCALE = 2
STEP_SIZE = 200
GAMMA = 0.5
EPOCHS = 50
START_EPOCH = 1
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
PATCH_SIZE = 12
isY = True
N_COLOURS = 3
RGB_RANGE = 1
TRAIN_ANNOTATIONS = 'data/metadata/train.csv'
VALID_ANNOTATIONS = 'data/metadata/valid.csv'

if(SCALE == 2):
    DOWNSAMPLED_DIR = 'data/downSampledImages'
else:
    DOWNSAMPLED_DIR = 'data/downSampledImages025'

In [3]:
transform = T.Compose([ #transforms to be used when iterating images
    T.ToTensor()
])

train_dataset = imageDataset(DOWNSAMPLED_DIR, TRAIN_ANNOTATIONS, N_COLOURS, RGB_RANGE, PATCH_SIZE, SCALE, transform)
valid_dataset = imageDataset(DOWNSAMPLED_DIR, VALID_ANNOTATIONS, N_COLOURS, RGB_RANGE, PATCH_SIZE, SCALE, transform)
valid_dataset.train = False

In [4]:
torch.backends.cudnn.benchmark = True
# random seed
#seed = args.seed
#if seed is None:
seed = random.randint(1, 10000)
print("Random Seed: ", seed)
random.seed(seed)
torch.manual_seed(seed)

Random Seed:  4320


In [5]:
print("===> Loading datasets")
train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=VALID_BATCH_SIZE, shuffle=False)

===> Loading datasets


In [6]:
if torch.cuda.is_available():
        device = "cuda"
else:
        device = "cpu"
print(f"using {device}")

using cpu


In [7]:
print("===> Building models")
model = esrt.ESRT(upscale = SCALE)
l1_criterion = nn.L1Loss()
entropyL1 = EntropyL1()

===> Building models
scale 0.16666666666666666
288
36


In [8]:
print("===> Setting GPU")
if(device == "cuda"):
    model = model.to(device)
    #l1_criterion = l1_criterion.to(device)
    entropyL1 = entropyL1.to(device)

===> Setting GPU


In [9]:
print("===> Setting Optimizer")
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

===> Setting Optimizer


In [10]:
def train(epoch):
    model.train()
    running_loss = 0.0
    utils.adjust_learning_rate(optimizer, epoch, STEP_SIZE, LEARNING_RATE, GAMMA)
    print('epoch =', epoch, 'lr = ', optimizer.param_groups[0]['lr'])
    for iteration, (lr_tensor, hr_tensor, _, _, entropy) in enumerate(train_loader, 1):

        if device=="cuda":
            lr_tensor = lr_tensor.to(device)  # ranges from [0, 1]
            hr_tensor = hr_tensor.to(device)  # ranges from [0, 1]

        optimizer.zero_grad()
        sr_tensor = model(lr_tensor)
        #loss_l1 = l1_criterion(sr_tensor, hr_tensor)
        #loss_sr = loss_l1
        loss_l1Entropy = entropyL1(sr_tensor, hr_tensor)
        loss_sr = loss_l1Entropy

        loss_sr.backward()
        optimizer.step()
        if iteration % 100 == 0:
            print("===> Epoch[{}]({}/{}): Loss_l1: {:.5f}".format(epoch, iteration, len(train_loader),
                                                                  loss_l1Entropy.item()))
        running_loss += loss_l1Entropy.item()
    average_loss = running_loss / len(train_loader)
    return average_loss

In [11]:
def valid(scale):
    model.eval()
    running_loss = 0.0
    avg_psnr, avg_ssim = 0, 0
    for batch in valid_loader:
        lr_tensor, hr_tensor, entropy = batch[0], batch[1], batch[4]
        if device=='cuda':
            lr_tensor = lr_tensor.to(device)
            hr_tensor = hr_tensor.to(device)

        with torch.no_grad():
            pre = model(lr_tensor)
        
        sr_tensor = model(lr_tensor)
        #loss_l1 = l1_criterion(sr_tensor, hr_tensor)
        loss_l1Entropy = entropyL1(sr_tensor, hr_tensor)
        sr_img = utils.tensor2np(pre.detach()[0])
        gt_img = utils.tensor2np(hr_tensor.detach()[0])
        crop_size = SCALE
        cropped_sr_img = utils.shave(sr_img, crop_size)
        cropped_gt_img = utils.shave(gt_img, crop_size)
        if isY is True:
            im_label = utils.quantize(sc.rgb2ycbcr(cropped_gt_img)[:, :, 0])
            im_pre = utils.quantize(sc.rgb2ycbcr(cropped_sr_img)[:, :, 0])
        else:
            im_label = cropped_gt_img
            im_pre = cropped_sr_img
        # print(im_pre.shape)
        # print(im_label.shape)
        avg_psnr += utils.compute_psnr(im_pre, im_label)
        avg_ssim += utils.compute_ssim(im_pre, im_label)
        running_loss += loss_l1Entropy.item()
    print("===> Valid. psnr: {:.4f}, ssim: {:.4f}".format(avg_psnr / len(valid_loader), avg_ssim / len(valid_loader)))
    return running_loss/len(valid_loader)

In [12]:
def save_checkpoint(epoch):
    model_folder = "experiment/checkpoint_ESRT_x{}/".format(SCALE)
    model_out_path = model_folder + "epoch_{}.pth".format(epoch)
    if not os.path.exists(model_folder):
        os.makedirs(model_folder)
    torch.save(model.state_dict(), model_out_path)
    print("===> Checkpoint saved to {}".format(model_out_path))

In [13]:
def print_network(net):
    num_params = 0
    for param in net.parameters():
        num_params += param.numel()
    # print(net)
    print('Total number of parameters: %d' % num_params)

In [14]:
print("===> Training")
print_network(model)
code_start = datetime.datetime.now()
timer = utils.Timer()
train_losses = []
val_losses = []
for epoch in range(START_EPOCH, EPOCHS + 1):
    t_epoch_start = timer.t()
    epoch_start = datetime.datetime.now()
    valid_loss = valid(SCALE)
    val_losses.append(valid_loss)
    train_loss = train(epoch)
    train_losses.append(train_loss)
    if epoch%10==0:
        save_checkpoint(epoch)
    epoch_end = datetime.datetime.now()
    print('Epoch cost times: %s' % str(epoch_end-epoch_start))
    t = timer.t()
    prog = (epoch-START_EPOCH+1)/(EPOCHS + 1 - START_EPOCH+ 1)
    t_epoch = utils.time_text(t - t_epoch_start)
    t_elapsed, t_all = utils.time_text(t), utils.time_text(t / prog)
    print('{} {}/{}'.format(t_epoch, t_elapsed, t_all))
code_end = datetime.datetime.now()
print('Code cost times: %s' % str(code_end-code_start))

===> Training
Total number of parameters: 677783
===> Valid. psnr: 11.5974, ssim: 0.2037
epoch = 1 lr =  0.0002
===> Epoch[1](100/200): Loss_l1: 0.14414
===> Epoch[1](200/200): Loss_l1: 0.09799
Epoch cost times: 0:04:15.808940
4.3m 4.3m/3.6h
===> Valid. psnr: 26.4901, ssim: 0.6404
epoch = 2 lr =  0.0002
===> Epoch[2](100/200): Loss_l1: 0.08690
===> Epoch[2](200/200): Loss_l1: 0.08742
Epoch cost times: 0:03:39.183929
3.7m 7.9m/3.4h
===> Valid. psnr: 28.6201, ssim: 0.7554
epoch = 3 lr =  0.0002
===> Epoch[3](100/200): Loss_l1: 0.08864
===> Epoch[3](200/200): Loss_l1: 0.05763
Epoch cost times: 0:03:39.811795
3.7m 11.6m/3.3h
===> Valid. psnr: 28.7209, ssim: 0.7785
epoch = 4 lr =  0.0002
===> Epoch[4](100/200): Loss_l1: 0.03330
===> Epoch[4](200/200): Loss_l1: 0.04696
Epoch cost times: 0:03:39.657426
3.7m 15.2m/3.2h
===> Valid. psnr: 30.0049, ssim: 0.8101
epoch = 5 lr =  0.0002
===> Epoch[5](100/200): Loss_l1: 0.06872
===> Epoch[5](200/200): Loss_l1: 0.05054
Epoch cost times: 0:03:37.004356

KeyboardInterrupt: 

In [ ]:
plt.plot(train_losses, label='train_loss')
plt.plot(val_losses,label='val_loss')
plt.legend()
plt.show

In [ ]:
from google.colab import files
model_folder = "experiment/checkpoint_ESRT_x{}/".format(SCALE)
model_out_path = model_folder + "epoch_{}.pth".format(epoch)
files.download(model_out_path)
